# **Purpose**
This notebook is a tutorial for ZapMeNot.

# **Configure for the Tutorial**
This step won't be required for production work.  The module search path is being altered to preferentially run
the "nightly builds" of both WatpropPy and ZapMeNot.

In [1]:
import sys
# sys.path.insert(0,'/home/alan025/bin/WatpropPy')
# sys.path.insert(0,'/home/alan025/bin/ZapMeNot')
sys.path.insert(0,'/Users/alan/Desktop/github/WatpropPy')
sys.path.insert(0,'/Users/alan/Desktop/github/ZapMeNot')



# **Import packages**
The Python packages required by this notebook are imported. The package and Python interpreter versions are printed to ensure analysis reproducibility. 

In [2]:
from IPython.display import display, Markdown, Math, Latex, Javascript, HTML
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
import math
from zap_me_not import model,source,shield,detector,material
import WatpropPy

# **Problem Descripton**
ESIS benchmark shielding problem \#1.

The source is a water-filled tank 108.3 cm high and 308 cm in diameter.
The tank wall is steel with a thickness of 2.54 cm.
Uniformly distributed in the water is the following source:
|  Energy<br>(MeV)     |     Source<br>(photons/sec/cm<sup>3</sup>) |
| :----:     |     :----:              |
|  0.4       |     4.0E+6            |
|  0.8       |     7.0E+6            |
|  1.3       |     2.8E+6            |
|  1.7       |     8.2E+6            |
|  2.2       |     4.0E+4            |
|  2.5       |     3.0E+4            |
|  3.5       |     1.2E+1            |
  
The  detector location is 311 cm from the tank centerline
at the vertical centerline of the tank.
A concrete shield wall between tank and detector starts at
x = 220 cm and ends at x=320 cm.
The water density is 1 g/cm<sup>3</sup> and the air density is 0.00122 g/cm3.
The the steel density is 7.8 g/cm<sup>3</sup>.
The concrete density is 2.4 g/cm<sup>3</sup>.  
The buildup material is concrete.

Benchmark acceptable results are 0.49E-3 to 2.46E-3 R/hr.


# **Model Construction**

In [3]:
ESIS_model = model.Model()
my_source = source.ZAlignedCylinderSource(material_name="water", cylinder_radius=154, \
       cylinder_center=[0,0,54.15], cylinder_length=108.3)
my_source.add_photon(0.4,3.2276e+013)
my_source.add_photon(0.8,5.6483e+013)
my_source.add_photon(1.3,2.2593e+013)
my_source.add_photon(1.7,6.6166e+013)
my_source.add_photon(2.2,3.2276e+011)
my_source.add_photon(2.5,2.4207e+011)
my_source.add_photon(3.5,9.6828e+007)
my_source.points_per_dimension = [16,16,16]
ESIS_model.add_source(my_source)
ESIS_model.add_shield(shield.ZAlignedInfiniteAnnulus("iron", cylinder_inner_radius=154, \
       cylinder_center=[0,0,54.15], cylinder_outer_radius=154+2.54, density=7.8))
ESIS_model.add_shield(shield.SemiInfiniteXSlab("concrete", x_start=220, \
       x_end=311, density=2.4))
ESIS_model.set_filler_material('air',density=0.00129)
ESIS_model.set_buildup_factor_material(material.Material('concrete'))
ESIS_model.add_detector(detector.Detector(311, 0, 54.15))

# **Calculate Dose Rate**

In [6]:
result = ESIS_model.calculate_exposure()
display(Markdown(f"**dose rate = {result:2.3} mR/hr**"))

**dose rate = 1.88 mR/hr**

# **Plot the Geometry**

In [5]:
ESIS_model.display()


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)